In [1]:
manhattandist(a,b) = sum(abs.(b-a))

function manhattandistmatrices(m,origin)
    #with wrap around taken into account
    Y=size(m,1)
    X=size(m,2)
    mhdist_q1=zeros(Int,(Y,X))
    mhdist_q2=zeros(Int,(Y,X))
    mhdist_q3=zeros(Int,(Y,X))
    mhdist_q4=zeros(Int,(Y,X))
    o=[origin[1],origin[2]]
    for x=1:X, y=1:Y
        mhdist_q1[y,x] = manhattandist([Y,1],[y,x])
        mhdist_q2[y,x] = manhattandist([Y,X],[y,x])
        mhdist_q3[y,x] = manhattandist([1,X],[y,x])
        mhdist_q4[y,x] = manhattandist([1,1],[y,x])
    end
    mhdist_q1 = shiftorigin(mhdist_q1, [Y,1])
    mhdist_q2 = shiftorigin(mhdist_q2, [Y,X])
    mhdist_q3 = shiftorigin(mhdist_q3, [1,X])
    mhdist_q4 = mhdist_q4
    
    return cat(mhdist_q1, mhdist_q2, mhdist_q3, mhdist_q4, dims=3)
end

manhattandistmatrices (generic function with 1 method)

In [64]:
rounddown(a) = round(a, RoundDown)

function quadrant_travelcost(m) 
    # cost: cheapest cost from top left corner to every other square restricted to only South/East moves. (quadrant 4)
    # and
    # dir: correct FIRST direction to go in order to take that cheapest path
    
    cost = m*0.1
    
    Y=size(m,1)
    X=size(m,2)
    dir=ones(Int,(Y,X))
    sumcost=zeros(Int,(Y,X))
    for x=1:X,y=1:Y
        if x>1
            sumcost[y,x] = sumcost[y,x-1] + rounddown(cost[y,x-1])
            dir[y,x]  = y==1 ? 1 : dir[y,x-1]
        end
        if y>1
            t_sumcost = sumcost[y-1,x] + rounddown(cost[y-1,x])
            if t_sumcost<sumcost[y,x] || x==1
                sumcost[y,x]=t_sumcost
                dir[y,x] = x==1 ? 0 : dir[y-1,x]
            end
        end
    end
    dir[1,1]=4
    
    return sumcost, dir
end

q1(m) = reverse(circshift(m, (-1,0)), dims=1)
q2(m) = q1(q3(m))
q3(m) = reverse(circshift(m, (0,-1)), dims=2)
q4(m) = m

iq1(m,d) = q1(m), q1(replace(d, 0 => 2))
iq2(m,d) = q2(m), q2(replace(d, 0 => 2, 1 => 3))
iq3(m,d) = q3(m), q3(replace(d, 1 => 3))
iq4(m,d) = q4(m), q4(d)

shiftorigin(m, origin) = circshift(m, (1-origin[1], 1-origin[2]))
ishiftorigin(m, origin) = circshift(m, (origin[1]-1, origin[2]-1))

function travelcost_3d(m, o)
    #Cheapest cost from some point to All Other points,
    #and what direction to go on First step to take that cheapest path.
    #south/east/north/west (in that order 0,1,2,3)
    m = shiftorigin(m, o)
    expand(f) = x -> f(x...)
    c1,d1 = m |> q1 |> quadrant_travelcost |> expand(iq1)
    c2,d2 = m |> q2 |> quadrant_travelcost |> expand(iq2)
    c3,d3 = m |> q3 |> quadrant_travelcost |> expand(iq3)
    c4,d4 = m |> q4 |> quadrant_travelcost |> expand(iq4)
    
    #3d
    D3 = cat(d1, d2, d3, d4, dims=3)
    C3 = cat(c1, c2, c3, c4, dims=3)
    MHD3 = manhattandistmatrices(m,o)
    return ishiftorigin(C3,o), ishiftorigin(D3,o), ishiftorigin(MHD3,o)
end


cheapest_C(C3) = findmin(C3, dims=3)[1][:,:,1]
#cheapest_D(C,D) = [D[y,x,findmin(C[y,x,:])[2]] for y=1:size(D,1), x=1:size(D,2)]

function travelcost(m,o)
    C3, D3, MHD3 = travelcost_3d(m, o)
    
    #make sure to pick the one with shortest distance in case cheapest cost is same for several.
    C = cheapest_C(C3)
    MHD = zeros(Int, size(C))
    D = zeros(Int, size(C))
    q = collect(1:4) #quartile number
    for x=1:size(C,2), y=1:size(C,1)
        i = q[C3[y,x,:] .== C[y,x]] #indexes with same minimum cost
        MHD[y,x], index = findmin(MHD3[y,x,i]) #pick minimum dist amongst the ones with least cost
        D[y,x] = D3[y,x,i][index] #pick the direction corresponding to that picked index
    end
    return C, D, MHD
end

travelcost (generic function with 1 method)

In [200]:
function halite_per_turn(m, p_ship, p_shipyard, ship_halite)
    #mined halite amount (at some point)
    #minus
    #cost (from ship, to some point, and then to shipyard)
    #divided by
    #nr turns to travel there and then to shipyard
    #equals
    #net halite gained per turn
    
    mining = ceil.(m .* 0.25)
    cost1, direction1, mhd1 = travelcost(m, p_ship)
    cost2, direction2, mhd2 = travelcost(m, p_shipyard)
    cost2 = cost2 + floor.(Int, (m - mining)*0.1)
    
    cost = cost1 + cost2
    net_gain = (mining - cost) .+ ship_halite
    net_gain[p_shipyard[1],p_shipyard[2]] = ship_halite - cost1[p_shipyard[1],p_shipyard[2]]
    
    mhd = mhd1 .+ mhd2
    #hpt = net_gain./mhd
    
    hpt = net_gain ./ (mhd.+1) #plus 1 since we mined one turn
    hpt[p_shipyard[1],p_shipyard[2]] = net_gain[p_shipyard[1],p_shipyard[2]] ./ mhd[p_shipyard[1],p_shipyard[2]]
    
    if p_ship == p_shipyard
        hpt[p_shipyard[1],p_shipyard[2]] = 0
    end
    
    return hpt, cost1, direction1, cost2
end

within_reach(hpt, cost1, ship_halite) = hpt.*(cost1 .<= ship_halite)

function select_direction(m, p_ship, p_shipyard, ship_halite)
    hpt, cost1, direction1 = halite_per_turn(m, p_ship, p_shipyard, ship_halite)
    hpt_within_reach = within_reach(hpt, cost1, ship_halite)
    dir = direction1[findmax(hpt_within_reach)[2]]
    return dir
end

select_direction (generic function with 1 method)

In [201]:
function manhattandistmatrices_johan(m, origin)
    s = [x + y - 2 for y in 1:size(m,1), x in 1:size(m,2)]
    return ishiftorigin(cat(q1(s), q2(s), q3(s), q4(s), dims=3), origin)
end

manhattandistmatrices_johan (generic function with 1 method)

In [202]:
manhattandistmatrices(m,p_ship)

4×4×4 Array{Int64,3}:
[:, :, 1] =
 0  1  2  3
 3  4  5  6
 2  3  4  5
 1  2  3  4

[:, :, 2] =
 0  3  2  1
 3  6  5  4
 2  5  4  3
 1  4  3  2

[:, :, 3] =
 0  3  2  1
 1  4  3  2
 2  5  4  3
 3  6  5  4

[:, :, 4] =
 0  1  2  3
 1  2  3  4
 2  3  4  5
 3  4  5  6

In [203]:
manhattandistmatrices_johan(m,p_ship)

4×4×4 Array{Int64,3}:
[:, :, 1] =
 4  1  2  3
 3  0  1  2
 6  3  4  5
 5  2  3  4

[:, :, 2] =
 2  1  4  3
 1  0  3  2
 4  3  6  5
 3  2  5  4

[:, :, 3] =
 4  3  6  5
 1  0  3  2
 2  1  4  3
 3  2  5  4

[:, :, 4] =
 6  3  4  5
 3  0  1  2
 4  1  2  3
 5  2  3  4

In [189]:
p_ship = [2,2]
p_shipyard = [4,4]
ship_halite = 0

m=[0 70 0 0;
    0 0 90 0;
    0 50 0 0;
    0 0 0 0]

m[p_shipyard[1],p_shipyard[2]]=0 #(make sure shipyard has 0 halite when constructing matrix manually)
m

4×4 Array{Int64,2}:
 0  70   0  0
 0   0  90  0
 0  50   0  0
 0   0   0  0

In [190]:
C, D, MHD = travelcost(m, p_ship)
C

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  5  0  0

In [191]:
D

4×4 Array{Int64,2}:
 3  2  3  3
 3  4  1  3
 3  0  3  3
 3  0  3  3

In [192]:
MHD

4×4 Array{Int64,2}:
 2  1  4  3
 1  0  1  2
 2  1  4  3
 3  2  5  4

In [204]:
hpt, cost1, direction1, cost2 = halite_per_turn(m, p_ship, p_shipyard, ship_halite)
hpt

4×4 Array{Float64,2}:
 0.6   3.2  0.428571  0.6 
 0.6   0.6  4.0       0.6 
 0.6   2.6  0.428571  0.6 
 0.6  -0.4  0.428571  0.75

In [205]:
cost1

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  5  0  0

In [206]:
direction1

4×4 Array{Int64,2}:
 3  2  3  3
 3  4  1  3
 3  0  3  3
 3  0  3  3

In [163]:
(90*0.25 -67.5*0.1)/5 # [2,3] should have value 3.15

3.15

In [164]:
cost2, direction2, mhd2 = travelcost(m, p_shipyard)
mhd2

4×4 Array{Int64,2}:
 2  3  2  1
 3  4  3  2
 2  3  2  1
 1  2  1  0

In [165]:
direction2

4×4 Array{Int64,2}:
 0  0  0  0
 2  2  2  2
 2  2  2  2
 1  1  3  4

In [166]:
cost2

4×4 Array{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0

In [185]:
(ceil(90*0.25) - floor((90-23)*0.1) + 3)/5

4.0

In [198]:
p_ship = [2,2]
p_shipyard = [4,4]
ship_halite = 3

m=[0 70 0 0;
    0 0 90 0;
    0 50 0 0;
    0 0 0 0]
select_direction(m, p_ship, p_shipyard, ship_halite)

1

In [199]:
hpt, cost1, direction1 = halite_per_turn(m, p_ship, p_shipyard, ship_halite)
hpt

4×4 Array{Float64,2}:
 0.6   3.2  0.428571  0.6 
 0.6   0.6  4.0       0.6 
 0.6   2.6  0.428571  0.6 
 0.6  -0.4  0.428571  0.75

In [168]:
m = [0 0 0 0
      0 0 0 0
      0 0 0 0
      0 0 10 0]

p_ship = [2,3]
p_shipyard = [2,2]
ship_halite= 3
select_direction(m, p_ship, p_shipyard, ship_halite)

3

In [175]:
hpt, cost1, direction1 = halite_per_turn(m, p_ship, p_shipyard, ship_halite)
hpt

4×4 Array{Float64,2}:
 0.5    0.75  0.75  0.5  
 0.75   3.0   1.5   0.75 
 0.5    0.75  0.75  0.5  
 0.375  0.5   1.0   0.375